In [1]:
# importing libraries
import pandas as pd

In [4]:
#loading datasets
df1 = pd.read_csv('./data/news_a1.csv')
print(df1.shape)
df1.head()

(8620, 7)


,Unnamed: 0.1,Unnamed: 0,title,text,subject,date,label
0,4218,15432,NEWSFLASH FOR OUR IMPERIAL PRESIDENT: STATES C...,As Barack Hussein Obama tours around the count...,politics,"Jul 23, 2015",FAKE
1,3828,19686,Factbox: Reactions to speech by Myanmar's Suu ...,NAYPYITAW (Reuters) - Myanmar leader Aung San ...,worldnews,"September 19, 2017",REAL
2,9288,10721,"RADICAL, INTOLERANT Students Held College Admi...",Video from inside Evergreen State College cont...,politics,"Jun 2, 2017",FAKE
3,11577,20031,Japan's Suga: government strongly protests lat...,TOKYO (Reuters) - North Korea fired a ballisti...,worldnews,"September 14, 2017",REAL
4,8869,10577,CNN’s Jim Acosta Goes Bonkers Waving His Hands...,Watch Jim Acosta wave his hands around and hav...,politics,"Jun 19, 2017",FAKE


In [5]:
df2 = pd.read_csv('./data/news_a2.csv')
print(df2.shape)
df2.head()

(8620, 7)


,Unnamed: 0.1,Unnamed: 0,title,text,subject,date,label
0,22406,4545,Trump officials defend immigration arrests at ...,LOS ANGELES (Reuters) - Federal agents have ar...,politicsNews,"March 31, 2017",REAL
1,4061,20039,LIBERAL SMACK DOWN OF THE DAY: Watch What Happ...,The Left is not able to get away with shaming ...,left-news,"Sep 2, 2016",FAKE
2,3456,6739,Factbox: Trump fills top jobs for his administ...,(Reuters) - U.S. President-elect Donald Trump ...,politicsNews,"December 13, 2016",REAL
3,4956,18869,"Iraq Kurdish vote may benefit Syrian Kurds, sa...",BEIRUT (Reuters) - The Iraqi Kurdish vote for ...,worldnews,"September 27, 2017",REAL
4,7787,17092,Tillerson to visit Pakistan as well as India: ...,WASHINGTON (Reuters) - U.S. Secretary of State...,worldnews,"October 18, 2017",REAL


In [10]:
df3 = pd.read_csv('./data/news2.csv')
print(df3.shape)
df3.head()

(8980, 6)


,Unnamed: 0,title,text,subject,date,label
0,22216,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",FAKE
1,4436,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",REAL
2,1526,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",REAL
3,1377,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",FAKE
4,8995,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",REAL


Data preparation and cleaning

In [15]:
# joining the datasets 
df = pd.concat([df1, df2, df3])
print(df.shape)
df.head()

(26220, 7)


,Unnamed: 0.1,Unnamed: 0,title,text,subject,date,label
0,4218.0,15432,NEWSFLASH FOR OUR IMPERIAL PRESIDENT: STATES C...,As Barack Hussein Obama tours around the count...,politics,"Jul 23, 2015",FAKE
1,3828.0,19686,Factbox: Reactions to speech by Myanmar's Suu ...,NAYPYITAW (Reuters) - Myanmar leader Aung San ...,worldnews,"September 19, 2017",REAL
2,9288.0,10721,"RADICAL, INTOLERANT Students Held College Admi...",Video from inside Evergreen State College cont...,politics,"Jun 2, 2017",FAKE
3,11577.0,20031,Japan's Suga: government strongly protests lat...,TOKYO (Reuters) - North Korea fired a ballisti...,worldnews,"September 14, 2017",REAL
4,8869.0,10577,CNN’s Jim Acosta Goes Bonkers Waving His Hands...,Watch Jim Acosta wave his hands around and hav...,politics,"Jun 19, 2017",FAKE


In [12]:
# checking for missing values
df.isnull().sum()

Unnamed: 0.1    8980
Unnamed: 0         0
title              0
text               0
subject            0
date               0
label              0
dtype: int64

In [17]:
# dropping duplicates
df.drop_duplicates(subset ="title", keep = 'first', inplace = True)
df.shape

(24051, 7)

In [18]:
# checking imbalanced data
df['label'].value_counts()

REAL    12303
FAKE    11748
Name: label, dtype: int64

Text Cleaning